In [ ]:
! pip install -r requirements.txt --quiet

# Creating and Deploying an Azure AI Agent Using SDK and MCP Tools

This notebook demonstrates how to create and deploy an Azure AI Agent using the Azure SDK, with tools generated from an OpenAPI endpoint. It provides a coded example that mirrors the portal example demonstrated in this workshop, offering a programmatic approach to deploying agents with OpenAPI-driven tools.


🔗 [How to use the Model Context Protocol tool](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol-samples)

In [ ]:
from os import environ
import time
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool
from openai.types.responses.response_input_param import McpApprovalResponse, ResponseInputParam


load_dotenv(override=True)

## Get MCP server configuration from environment variables

In [ ]:
mcp_server_url = environ["MCP_SERVER_URL"]
mcp_server_label = environ["MCP_SERVER_LABEL"] 

## Create a project client

In [ ]:
# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

openai_client = project_client.get_openai_client()

## Initialize agent MCP tool

In [ ]:

audit_mcp_tool = MCPTool(
        server_label=mcp_server_label,
        server_url=mcp_server_url,
        require_approval="never",  # set to "never" for smoother experience during testing

    )


## Create an agent and a thread

In [ ]:
instructions = """
You are an audit assistant supporting Accounts Payable audit procedures.

You have two authoritative sources, accessed ONLY via tools:
1) Audit Operational Data (MCP/Cosmos): invoices, vendors, payments. This is the ONLY source of truth for operational facts.
2) Audit Policy Library (audit-poc): company audit policy documents (RAG). This is the ONLY source of truth for policy language and criteria. Do not invent policy text or citations if not found.

Rules
- Do not invent records or values. If a tool did not return it, you do not have it.
- Do not use policy text as evidence that a specific invoice/payment violated policy. Use Cosmos records as evidence; use policy snippets as criteria.
- Cite IDs for every operational claim:
  - Invoices → invoice_id(s)
  - Vendors → vendor_id(s)
  - Payments → payment_id(s)
- Cite policy references with policy_id + section_id (or section) + effective_date when available.

Tool selection
- Discovery/listing (Q3 invoices, duplicates, missing PO/receipt, high-risk vendors):
  - Start with query_invoices and/or query_payments, then use get_invoice/get_vendor/get_payment as needed.
- ID-based lookup (exact invoice/vendor/payment ID provided):
  - Use get_invoice, get_vendor, get_payment, or get_payments_for_invoice.
- Policy questions (duplicates, three-way match, vendor governance):
  - Use the policy search/retrieval tools (Azure AI Search) before answering.

Output
- Keep responses audit-ready and structured.
- For findings, include:
  1) Evidence (Cosmos fields + cited IDs)
  2) Policy criteria (policy citation)
  3) Conclusion (clearly labeled; do not overstate certainty)
- If required data is missing, state what is missing and which tool call is needed next.

"""

In [ ]:


agent = project_client.agents.create_version(
    agent_name="AuditAgent",
    definition=PromptAgentDefinition(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        instructions=instructions,
        tools=[audit_mcp_tool],
    ),
)
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

conversation = openai_client.conversations.create()
print(f"Created conversation (id: {conversation.id})")


In [ ]:
# Send a request that should trigger MCP tool calls
# (The MCP server expose tools like query_invoices/get_invoice/get_vendor/query_payments)
response = openai_client.responses.create(
    conversation=conversation.id,
    input=(
        "Identify potential duplicate invoices in Q3 2025 where the same vendor_id and amount occur within 7 days."
    ),
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)


In [ ]:
input_list: ResponseInputParam = []
for item in response.output:
    if item.type == "mcp_approval_request" and item.server_label == "audit-mcp" and item.id:
        input_list.append(
            McpApprovalResponse(
                type="mcp_approval_response",
                approve=True,
                approval_request_id=item.id,
            )
        )

# If approvals were needed, send them back to continue execution
if input_list:
    response = openai_client.responses.create(
        input=input_list,
        previous_response_id=response.id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )

print(f"Response: {response.output_text}")

In [ ]:
project_client.agents.delete_version(agent_name=agent.name, agent_version=agent.version)
print("Agent deleted")